In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.cuda.amp as amp

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

#importing distributed and weights and biases
import torch.distributed as dist
import wandb

In [2]:
wandb.init(project='Homework9-take2', entity='malachyiii')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: malachyiii (use `wandb login --relogin` to force relogin)



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [3]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
torch.cuda.device(0)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

True
0
1
Tesla T4


In [4]:
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [5]:
#Trying resnet34 because it is fast
ARCH = torchvision.models.resnet34(pretrained=False)
START_EPOCH = 0
EPOCHS = 3

#The following parameters based on the paper at https://arxiv.org/pdf/1512.03385.pdf
TRAIN_BATCH=256
VAL_BATCH=256
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
LR = 0.01
IMG_SIZE = 224

#Logging wandb config

wandb.config = {
  "learning_rate": LR,
  "epochs": EPOCHS,
  "batch_size": TRAIN_BATCH,
  "momentum": MOMENTUM,
  "weight_decay": WEIGHT_DECAY,
  "image_size": IMG_SIZE
}

PRINT_FREQ = 50
WORKERS=2

TRAINDIR="/data/train"
VALDIR="/data/val"

In [6]:
GPU = torch.cuda.current_device()
torch.cuda.device(GPU)
cudnn.benchmark = True

In [7]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [8]:
#Setting up the cluster and the world
WORLD_SIZE = 2
BACKEND = 'nccl'
URL = 'tcp://35.163.183.149:5000'
RANK = 0

dist.init_process_group(backend = BACKEND, init_method= URL,
                                world_size= WORLD_SIZE, rank=RANK)

In [9]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [10]:
transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # transformations based on https://arxiv.org/pdf/1512.03385.pdf
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

train_dataset = torchvision.datasets.ImageFolder('/data/train', transform=transform_train)
val_dataset = torchvision.datasets.ImageFolder('/data/val', transform=transform_val)

In [11]:
#Setting up the train loader with a distributed Sampler
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH, 
                              shuffle=False, num_workers=WORKERS, pin_memory=True, 
                                           sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=VAL_BATCH, 
                              shuffle=False, num_workers=WORKERS, sampler=None)

In [12]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))
    
    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        if GPU is not None:
            images, target = images.cuda(GPU, non_blocking = True), target.cuda(GPU, non_blocking = True)
        # send the target to cuda device

        
        ####Utilizing PyTorch native AMP####
        with amp.autocast():
            # compute output
            output = model(images)
            # compute loss 
            loss = criterion(output, target)
        
        
        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        #Logging the metrics
        wandb.log({"loss": loss,
                   "acc1": acc1,
                   "acc5": acc5})
        
        wandb.watch(model)

        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        optimizer.zero_grad()
        
        ## backprop!
        scaler.scale(loss).backward()
        
        # update the weights!
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [13]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            images, target = images.to(GPU), target.to(GPU)

            # compute output
            output = model(images)

            # compute loss
            loss = criterion(output, target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [14]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    torch.save(state, filename) 
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [16]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [17]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [18]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 1/3 of the epochs"""
    lr = LR * (0.1 ** (epoch // int(round(EPOCHS/3, 0))))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [19]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [20]:
model = ARCH
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [21]:
criterion = nn.CrossEntropyLoss().cuda(GPU)
optimizer = torch.optim.SGD(params = model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
#Based on the same paper, https://arxiv.org/pdf/1512.03385.pdf
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2])
scaler = amp.GradScaler()

In [22]:
best_acc1 = 0

In [23]:
#Initiating weights and biases
wandb.watch(model)

for epoch in range(START_EPOCH, EPOCHS):
    #adjust_learning_rate(optimizer, epoch)
    
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step(acc1)
    
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][   0/2503]	Time  9.991 ( 9.991)	Data  2.178 ( 2.178)	Loss 7.0532e+00 (7.0532e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.78 (  0.78)


Epoch: [0][  50/2503]	Time  1.503 ( 1.192)	Data  0.980 ( 0.391)	Loss 6.8969e+00 (6.9580e+00)	Acc@1   0.00 (  0.14)	Acc@5   0.78 (  0.66)


Epoch: [0][ 100/2503]	Time  1.179 ( 1.121)	Data  0.653 ( 0.407)	Loss 6.8353e+00 (6.9162e+00)	Acc@1   0.00 (  0.19)	Acc@5   0.78 (  0.84)


Epoch: [0][ 150/2503]	Time  1.471 ( 1.099)	Data  0.799 ( 0.400)	Loss 6.7175e+00 (6.8811e+00)	Acc@1   0.00 (  0.23)	Acc@5   1.95 (  1.06)


Epoch: [0][ 200/2503]	Time  1.611 ( 1.079)	Data  0.935 ( 0.413)	Loss 6.6621e+00 (6.8423e+00)	Acc@1   1.56 (  0.32)	Acc@5   2.34 (  1.35)


...

Test: [ 50/196]	Time  2.268 ( 1.234)	Loss 3.4507e+00 (3.4355e+00)	Acc@1  23.05 ( 25.84)	Acc@5  48.83 ( 52.08)


Test: [100/196]	Time  1.575 ( 1.223)	Loss 4.9146e+00 (3.6011e+00)	Acc@1   9.77 ( 24.50)	Acc@5  21.48 ( 49.20)


Test: [150/196]	Time  1.393 ( 1.178)	Loss 4.5324e+00 (3.7452e+00)	Acc@1  16.41 ( 23.11)	Acc@5  31.64 ( 46.54)


 * Acc@1 23.008 Acc@5 46.276


lr: [0.001]
